In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import configparser
import os.path
from os import path
from importlib import reload
import wandb


creds_path_ar = ["../../credentials.ini","credentials.colab.ini"]
PATH_ROOT = ""
PATH_DATA = ""

for creds_path in creds_path_ar:
    if path.exists(creds_path):
        config_parser = configparser.ConfigParser()
        config_parser.read(creds_path)
        PATH_ROOT = config_parser['MAIN']["PATH_ROOT"]
        PATH_DATA = config_parser['MAIN']["PATH_DATA"]
        WANDB_enable = config_parser['MAIN']["WANDB_ENABLE"] == 'TRUE'
        ENV = config_parser['MAIN']["ENV"]
        break

if ENV=="COLAB":
  from google.colab import drive
  mount_path = '/content/gdrive/'
  drive.mount(mount_path)

In [4]:
if WANDB_enable == True:
    wandb.init(project="sota-mafat-base")
    os.environ['WANDB_NOTEBOOK_NAME'] = '[SS]TCN_pytorch'

In [5]:
cd {PATH_ROOT}

/home/shaul/workspace/GitHub/sota-mafat-radar


In [67]:
import os
import random
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim


from sklearn.metrics import roc_auc_score, roc_curve, auc, accuracy_score
from matplotlib.colors import LinearSegmentedColormap
from termcolor import colored

from src.data import feat_data, get_data
from src.models import arch_setup, base_base_model, alex_model, tcn_model

# Set seed for reproducibility of results
seed_value = 0
os.environ['PYTHONHASHSEED']=str(seed_value)


random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(0)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu:0')

In [119]:
config = dict()
config['num_tracks'] = 3
config['val_ratio'] = 6
config['shift_segment'] = [4,8,12,16,20]
config['get_shifts'] = True
config['get_horizontal_flip'] = False
config['get_vertical_flip'] = False

In [120]:
train_x, train_y, val_x, val_y = get_data.classic_trainval(PATH_ROOT+PATH_DATA)

Everything is setup correctly


In [121]:
train_set = arch_setup.DS(train_x.squeeze(),train_y)
val_set= arch_setup.DS(val_x.squeeze(),val_y)

train_loader=DataLoader(dataset= train_set, batch_size = 32, shuffle = True, num_workers = 2)
val_loader=DataLoader(dataset= val_set, batch_size = 32, shuffle = True, num_workers = 2)

In [122]:
model = tcn_model.TCNModel(num_channels=[64]*4, kernel_size=3, dropout=0.25)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.to(device)

if WANDB_enable == True:
    wandb.watch(model)

In [123]:
log = arch_setup.train_epochs(train_loader,val_loader,model,criterion,optimizer,num_epochs= 10,device=device,train_y=train_y,val_y=val_y, WANDB_enable = WANDB_enable, wandb= wandb)

started training epoch no. 1
epoch : 1.0000, loss : 57.4072, auc : 0.8882, acc : 0.8797, val_loss : 7.5496, val_auc : 0.7566, val_acc : 0.6084, 
---------------------------

started training epoch no. 2
epoch : 2.0000, loss : 45.9995, auc : 0.9335, acc : 0.9038, val_loss : 6.1219, val_auc : 0.7696, val_acc : 0.6926, 
---------------------------

started training epoch no. 3
epoch : 3.0000, loss : 42.6790, auc : 0.9433, acc : 0.9117, val_loss : 7.1849, val_auc : 0.7556, val_acc : 0.6505, 
---------------------------

started training epoch no. 4
epoch : 4.0000, loss : 38.6706, auc : 0.9547, acc : 0.9207, val_loss : 7.7564, val_auc : 0.7666, val_acc : 0.6570, 
---------------------------

started training epoch no. 5
epoch : 5.0000, loss : 35.4645, auc : 0.9626, acc : 0.9306, val_loss : 6.2351, val_auc : 0.7950, val_acc : 0.7152, 
---------------------------

started training epoch no. 6
epoch : 6.0000, loss : 31.7643, auc : 0.9713, acc : 0.9373, val_loss : 6.2097, val_auc : 0.7998, val_

In [ ]:
arch_setup.plot_loss_train_test(log,model)

In [ ]:
arch_setup.plot_ROC(train_x,val_x,train_y,val_y,model,device)